In [1]:
import pdb
import tensorflow as tf

from tensorflow import keras
from datetime import datetime
import matplotlib.pyplot as plt
#import tensorflow_addons as tfa
from itertools import combinations
from tensorflow.keras import layers
#import tensorflow_probability as tfp
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Concatenate
#tf.config.experimental_run_functions_eagerly(True)

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape, Lambda, Conv2DTranspose, UpSampling2D
from tensorflow.keras.models import Model

In [3]:
latent_dim = 32

In [25]:

encoder_input = keras.Input(shape=(128, 128, 1))

x = layers.Conv2D(filters=8, kernel_size=(3,5), strides=(2,2), activation=None, padding='same')(encoder_input)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=11, strides=(4,2), activation=None, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(filters=32, kernel_size=11, strides=(4,2), activation=None, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(filters=64, kernel_size=11, strides=(4,4), activation=None, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Flatten()(x)
#x = layers.Dropout(0.3)(x)
#x = layers.Dense(64, activation="relu")(x)

z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name="encoder")

# Decoder

latent_inputs = keras.Input(shape=(latent_dim,))

#x = layers.Dropout(0.5)(latent_inputs)
dec = layers.Dense(units=1*4*64, activation="relu")(latent_inputs)
#x = layers.Dropout(0.5)(x)
dec = layers.Reshape(target_shape=(1,4,64))(dec)
dec = layers.UpSampling2D(size=(4, 4))(dec)
dec = layers.Conv2D(filters=32, kernel_size=11, strides=1, padding='same', activation=None)(dec)
dec = layers.BatchNormalization()(dec)
dec = layers.ReLU()(dec)
dec = layers.UpSampling2D(size=(4, 2))(dec)
dec = layers.Conv2D(filters=16, kernel_size=11, strides=1, padding='same', activation=None)(dec)
dec = layers.BatchNormalization()(dec)
dec = layers.ReLU()(dec)
dec = layers.UpSampling2D(size=(4, 2))(dec)
dec = layers.Conv2D(filters=8, kernel_size=11, strides=1, padding='same', activation=None)(dec)
dec = layers.BatchNormalization()(dec)
dec = layers.ReLU()(dec)
dec = layers.UpSampling2D(size=(2, 2))(dec)
decoder_outputs = layers.Conv2D(filters=1, kernel_size=(3,5), strides=1, padding='same', activation='relu')(dec)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# Define VAE

decoder_output = decoder(encoder(encoder_input))
model = Model(encoder_input, decoder_output)

In [27]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               8448      
_________________________________________________________________
reshape_11 (Reshape)         (None, 1, 4, 64)          0         
_________________________________________________________________
up_sampling2d_24 (UpSampling (None, 4, 16, 64)         0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 4, 16, 32)         247840    
_________________________________________________________________
batch_normalization_136 (Bat (None, 4, 16, 32)         128       
_________________________________________________________________
re_lu_136 (ReLU)             (None, 4, 16, 32)         0   

In [26]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 64, 64, 8)         128       
_________________________________________________________________
batch_normalization_132 (Bat (None, 64, 64, 8)         32        
_________________________________________________________________
re_lu_132 (ReLU)             (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 16, 32, 16)        15504     
_________________________________________________________________
batch_normalization_133 (Bat (None, 16, 32, 16)        64        
_________________________________________________________________
re_lu_133 (ReLU)             (None, 16, 32, 16)        0   